In [1]:
import numpy as np 
import time 
import numpy as np
import torch
import torch.nn as nn
import time
import gym 
import pickle  
import argparse
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions.transformed_distribution import TransformedDistribution
from torch.distributions.transforms import TanhTransform
import tqdm 

### TODO: Fix training issue, Doesn't work well

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
EPS = 1e-7

In [4]:
class Memory: 
    def __init__(self, maxlen=1_000_000):  
        self.states=None
        self.actions=None
        self.ids=None
        self.maxlen=maxlen
        
    def add_data(self, states, actions):
        """
        add N s,a,r
        """
        if self.states is None: #first time
            self.states =states[-self.maxlen:] 
            self.actions=actions[-self.maxlen:]
        else:
            self.states = np.concatenate([self.states, states])[-self.maxlen:] 
            self.actions = np.concatenate([self.actions, actions])[-self.maxlen:] 
        
    def sample_batch(self, batch_size):
        indices = np.random.permutation(len(self))[:batch_size]
        batch_states=self.states[indices] 
        batch_actions=self.actions[indices]
   
        return batch_states, batch_actions 
        
    def __len__(self):
        return len(self.states)

In [5]:
class Actor(nn.Module):
    def __init__(self, action_dim, state_dim, size=32):
        super(Actor, self).__init__()

        self.mlp = nn.ModuleList()
        self.mlp.append(nn.Linear(state_dim, size))
        self.mlp.append(nn.Tanh())

        # self.mlp.append(nn.Linear(size, size))
        # self.mlp.append(nn.Tanh()) 

        self.mu_head = nn.Linear(size, action_dim)
        self.sigma_head = nn.Linear(size, action_dim)

    def _get_outputs(self, x):
        for layer in self.mlp:
            x = layer(x)
        
        mu = self.mu_head(x)
        # mu = torch.clip(mu, MEAN_MIN, MEAN_MAX)
        log_sigma = self.sigma_head(x)
        # log_sigma = torch.clip(log_sigma, LOG_STD_MIN, LOG_STD_MAX)
        sigma = torch.exp(log_sigma)

        a_distribution = TransformedDistribution(
            Normal(mu, sigma), TanhTransform(cache_size=1)
        )
        a_tanh_mode = torch.tanh(mu)
        return a_distribution, a_tanh_mode
    
    def get_mean(self, x):
        for layer in self.mlp:
            x = layer(x)
        
        mu = self.mu_head(x)
        return mu
    
    def get_action(self, x, deterministic=False):
        a_dist, a_tanh_mode = self._get_outputs(x)
        if deterministic:
            action = a_tanh_mode
        else:
            action = a_dist.rsample()
        return action
    
    def forward(self, state):
        a_dist, a_tanh_mode = self._get_outputs(state)
        action = a_dist.rsample()
        logp_pi = a_dist.log_prob(action).sum(axis=-1)
        return action, logp_pi, a_tanh_mode

    def get_log_density(self, state, action):
        a_dist, _ = self._get_outputs(state)
        action_clip = torch.clip(action, -1. + EPS, 1. - EPS)
        logp_action = a_dist.log_prob(action_clip)
        return logp_action


In [6]:
def play(env, actor, is_close=True, is_render=True, max_step=1000): 
    obs,info = env.reset()
    dones=False
    total_r=0
    step=0
    while not dones: 
        step+=1
        obs=torch.Tensor(obs[None]).to(device)
        action, logp_pi, a_tanh_mode = actor(obs)
        # action=actor.get_mean(obs)
        action=action.cpu().detach().numpy() 

        obs, rewards, done, s, info = env.step(action.ravel())
        total_r +=rewards  
        if done:
            break
        if step>max_step:
            # print('max step reached')
            break
        # elif s:
        #     print('solved!')
        #     break
    if is_close:
        env.close()
    return {'reward':total_r, 'step':step-1}

In [7]:
env_name='Pendulum-v1'
data_path="/home/ns/bc_tutorial/pendulum/expert_data/Pendulum-v1_10_-130.pkl" 
data_path = "expert_data/Pendulum-v1_50_-149.pkl"

In [8]:
with open(data_path, "rb") as f:
    data_good = pickle.load(f)
print('expert data loaded')

data_good=data_good[:20]

good_obs=[]
good_acts=[]
good_rews=[]

for traj in data_good: 
    s,a,r=traj  

    good_obs.append(s)
    good_acts.append(a)
    good_rews.append(r)

good_obs=np.vstack(good_obs)
good_acts=np.vstack(good_acts)
good_rews=np.vstack(good_rews) 

good_obs.shape, good_acts.shape, good_rews.shape

expert data loaded


((4000, 3), (4000, 1), (4000, 1))

In [9]:
np.min(good_acts), np.max(good_acts), np.mean(good_acts), np.std(good_acts)

(-1.9999993, 1.9999466, -0.29554793, 0.7047518)

In [10]:
action_min=np.min(good_acts)
action_max=np.max(good_acts)

#convert to -1,1
actions= -1 + (good_acts-action_min)/(action_max-action_min)*2

np.min(actions), np.max(actions), np.mean(actions), np.std(actions)

(-1.0, 1.0, -0.1477628, 0.35238066)

In [11]:
action_min, action_max

(-1.9999993, 1.9999466)

In [12]:
states=good_obs
actions=actions

In [13]:
memory=Memory(maxlen=1000000)
memory.add_data(states, actions)

In [14]:
seed=0
torch.manual_seed(seed)
np.random.seed(seed) 
env = gym.make(env_name) 

ob_dim = env.observation_space.shape[0]
ac_dim = env.action_space.shape[0]

actor = Actor(ac_dim, ob_dim).to(device)
 
optimizer = torch.optim.Adam(actor.parameters(), 0.0001)

In [15]:
verbose=True
batch_size=100
training_timestep=10000
actor.train()
for train_step in range(1, training_timestep+1):  
    batch_states, batch_actions =memory.sample_batch(batch_size)  
    batch_states = torch.Tensor(batch_states).to(device)
    batch_actions=torch.Tensor(batch_actions).to(device)
     
    log_pi_e = actor.get_log_density(batch_states, batch_actions) 
    bc_loss = -torch.sum(log_pi_e, 1)
    loss = torch.mean(bc_loss)
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if verbose and (train_step % (training_timestep/10)==0):
        print('step:', train_step, ' loss', loss.item()) 

print('bc training completed')

step: 1000  loss 0.26415225863456726
step: 2000  loss 0.3189757168292999
step: 3000  loss -0.06276030838489532
step: 4000  loss -0.3269127607345581
step: 5000  loss -0.22007445991039276
step: 6000  loss -0.0013023471692577004
step: 7000  loss -0.36797699332237244
step: 8000  loss -0.26837703585624695
step: 9000  loss -0.2672169506549835
step: 10000  loss -0.2926677167415619
bc training completed


In [28]:
train_x=torch.tensor(states, dtype=torch.float)
train_y=torch.tensor(actions, dtype=torch.float)

In [29]:
yhat=actor.get_mean(train_x.to(device)).cpu().detach().numpy()

action_unnorm = action_min + (yhat + 1.) *  (action_max - action_min) / (1+1)

yhat=action_unnorm
score = 1 - ( (actions - yhat )**2 ).sum() /( (actions - actions.mean() )**2 ).sum()
print('regression score: ', score)

regression score:  -4.046102046966553


In [16]:
env = gym.make(env_name)
obs,info = env.reset()

In [17]:
obs=torch.Tensor(obs[None]).to(device)
action, logp_pi, a_tanh_mode = actor(obs)
# action=actor.get_mean(obs)
action=action.cpu().detach().numpy() 
action

array([[-1.]], dtype=float32)

In [18]:
actor.get_mean(obs)

tensor([[1.8385]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [19]:
action=actor.get_action(obs, deterministic=True)[0][0].cpu().detach().numpy()
action_unnorm = action_min + (action + 1.) *  (action_max - action_min) / (1+1)
action_unnorm

1.901257300542163

In [20]:
def play(env, actor, is_close=True, is_render=True, max_step=1000): 
    obs,info = env.reset()
    dones=False
    total_r=0
    step=0
    while not dones: 
        step+=1
        obs=torch.Tensor(obs[None]).to(device)
        action=actor.get_action(obs, deterministic=True)[0][0].cpu().detach().numpy()
        action_unnorm = action_min + (action + 1.) *  (action_max - action_min) / (1+1)
        action=action_unnorm

        obs, rewards, done, trunc, info = env.step([action] )
        total_r +=rewards  
        if done or trunc:
            break
    return {'reward':total_r, 'step':step-1}

In [21]:
play(env, actor, is_close=True, is_render=False)

{'reward': -1176.6829153347444, 'step': 199}

In [22]:
rewards=[]
for i in tqdm.tqdm( range(50) ):
    stats=play(env, actor, is_close=True, is_render=False)
    rewards.append(stats['reward'])
    # print(f'i={i} reward={stats["reward"]} step={stats["step"]}')
print(f'mean reward={np.mean(rewards) :.2f} std={np.std(rewards):.2f}')

100%|██████████| 50/50 [00:06<00:00,  7.43it/s]

mean reward=-1332.46 std=184.98
